<a href="https://colab.research.google.com/github/ViniciusBerti/20241_maua_ecm252_intro_git/blob/main/IMT_P3_2025_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Avaliação P3**: Regressão e Classificação, Métricas

Explore os dados abaixo e a seguir implemente modelos de regressão e classificação.

In [2]:
#@markdown run before
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
path = 'https://github.com/Rogerio-mack/IMT_CD_2025/raw/refs/heads/main/data/data3/'

In [3]:
df = pd.read_csv(path + 'credit_card_data.csv')
print("Dados para o aprendizado do modelo...")
display(df.head())
print()

df_case_regression = pd.read_csv(path + 'credit_card_data_case_regression.csv')
print("Dados para predição de `expenditure`")
display(df_case_regression.head())
print()

df_case_classification = pd.read_csv(path + 'credit_card_data_case_classification.csv')
print("Dados para predição de `card`")
display(df_case_classification.head())
print()



Dados para o aprendizado do modelo...


,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active,client_name,sex
0,high,0,37,4.52,0.03,124.98,yes,no,3,54.0,1,12,Ivan Gomez,Male
1,high,0,33,2.42,0.01,9.85,no,no,3,34.0,1,13,Sean Williams,Not Declared
2,high,0,33,4.50,0.00,15.00,yes,no,4,58.0,1,5,William Beard,Male
3,high,0,30,2.54,0.07,137.87,no,no,0,25.0,1,7,Cynthia Cuevas,Female
4,high,0,32,9.79,0.07,546.50,yes,no,2,64.0,1,5,Julie Davis,Female



Dados para predição de `expenditure`


,card,reports,age,income,share,owner,selfemp,dependents,months,majorcards,active,client_name,sex
0,high,0,32,3.37,0.07,no,no,0,55,0,6,Gary Williams,Male
1,high,0,25,2.24,0.00,no,no,0,12,1,2,Jill Flores,Female
2,high,1,46,5.50,0.18,no,no,3,143,1,16,Johnny Jones,Male



Dados para predição de `card`


,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active,client_name,sex
0,0,32,3.37,0.07,185.06,no,no,0,55.0,0,6,Jason Ramirez,Male
1,0,25,2.24,0.00,4.58,no,yes,0,NaN,1,2,Eric Simon,Not Declared
2,1,30,2.18,0.00,0.00,no,yes,0,13.0,1,1,Julian Choi,Male


# Exercício 1. Regressão.

Faça um modelo de regressão linear para estimar novos valores de `expenditure` a partir dos **demais atributos preditores** que sejam **significativos**. Para atributos categóricos considere significativos **somente atributos em que todos os encodes forem significativos**.

- **Valores ausentes** deve ser substituídos por valores com a mesma distribuição dentro do gênero (sexo). Empregue o seed=42 para seleção aleatória de valores a serem imputados.

Em seguida faça um modelo robusto indicado e compare os resultados.



## 1.1. Qual o $R2$ obtido pelo modelo de regressão linear somente com os atributos **preditores significativos**?



In [9]:
import pandas as pd, statsmodels.api as sm

df = pd.read_csv(path + 'credit_card_data.csv')
df = pd.get_dummies(df.drop(columns=['client_name']), drop_first=True).dropna()

y = df['expenditure'].astype(float)
X = sm.add_constant(df.drop(columns=['expenditure']).astype(float))

model = sm.OLS(y, X).fit()
print(round(model.rsquared, 3))

0.812


## 1.2. Qual a predição de despesas (`expenditure`) dos clientes da base `credit_card_data_case_regression`?

In [14]:
df = pd.read_csv(path + 'credit_card_data.csv')
df_case = pd.read_csv(path + 'credit_card_data_case_regression.csv')

df_case_id = df_case['client_name']

df = pd.get_dummies(df.drop(columns=['client_name']), drop_first=True)
df_case = pd.get_dummies(df_case.drop(columns=['client_name']), drop_first=True)

df_case = df_case.reindex(columns=df.drop(columns=['expenditure']).columns, fill_value=0)

df = df.fillna(0)
df_case = df_case.fillna(0)

y = df['expenditure'].astype(float)
X = sm.add_constant(df.drop(columns=['expenditure']).astype(float))

model = sm.OLS(y, X).fit()

X_case = sm.add_constant(df_case).astype(float)
preds = model.predict(X_case)

result = pd.DataFrame({'client_name': df_case_id, 'pred_expenditure': preds.round(2)})
print(result)

     client_name  pred_expenditure
0  Gary Williams            193.78
1    Jill Flores            -55.19
2   Johnny Jones            585.02


## 1.3.A Você pode notar que a previsão de despesas de `Jill Flores` foi negativa. Não faz sentido um valor de despesa negativo. Como você pode corrigir isso? Qual a nova predição da despesa de `Jill Flores`?

In [41]:

df_train = pd.read_csv(path + 'credit_card_data.csv')
y = df_train['expenditure']
X = df_train.drop(columns=['expenditure', 'client_name'])

X = pd.get_dummies(X, drop_first=True)

df_case = pd.read_csv(path + 'credit_card_data_case_regression.csv')
case_names = df_case['client_name']
X_case = pd.get_dummies(df_case.drop(columns=['client_name'], errors='ignore'), drop_first=True)

X_case = X_case.reindex(columns=X.columns, fill_value=0)

X = X.fillna(0)
X_case = X_case.fillna(0)

reg = LinearRegression().fit(X, y)

pred = reg.predict(X_case)
pred_corr = np.maximum(pred, 0)

result = pd.DataFrame({
    'client_name': case_names,
    'pred_original': np.round(pred, 2),
    'pred_corrigido': np.round(pred_corr, 2)
})

print(result[result['client_name'] == 'Jill Flores'])

   client_name  pred_original  pred_corrigido
1  Jill Flores         -55.19             0.0


## 1.4A Empregue agora um modelo equivalente (mesmos regressores) mais robusto, o `Huber Regressor`. Qual o seu **$R2$**? É maior que o anterior? Qual o valor de despesa de `Jill Flores` nesse novo modelo?

In [21]:
import pandas as pd
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import r2_score

df = pd.read_csv(path + 'credit_card_data.csv')
df_case = pd.read_csv(path + 'credit_card_data_case_regression.csv')

df_case_id = df_case['client_name']

df = pd.get_dummies(df.drop(columns=['client_name']), drop_first=True)
df_case = pd.get_dummies(df_case.drop(columns=['client_name']), drop_first=True)

df_case = df_case.reindex(columns=df.drop(columns=['expenditure']).columns, fill_value=0)

df = df.fillna(0)
df_case = df_case.fillna(0)

y = df['expenditure']
X = df.drop(columns=['expenditure'])

huber = HuberRegressor().fit(X, y)

r2 = r2_score(y, huber.predict(X))

preds = huber.predict(df_case)

result = pd.DataFrame({'client_name': df_case_id, 'pred_expenditure': preds.round(2)})
print("R² (Huber):", round(r2, 3))
print(result)

R² (Huber): 0.346
     client_name  pred_expenditure
0  Gary Williams            211.29
1    Jill Flores            177.72
2   Johnny Jones            299.46


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_huber.py:343: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## 1.4B Empregue agora um modelo equivalente (mesmos regressores) mais robusto, o `Ridge Regressor`. Qual o seu **$R2$**? É maior que o anterior? Qual o valor de despesa de `Jill Flores` nesse novo modelo?

In [17]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

df = pd.read_csv(path + 'credit_card_data.csv')
df_case = pd.read_csv(path + 'credit_card_data_case_regression.csv')

df_case_id = df_case['client_name']

df = pd.get_dummies(df.drop(columns=['client_name']), drop_first=True)
df_case = pd.get_dummies(df_case.drop(columns=['client_name']), drop_first=True)

df_case = df_case.reindex(columns=df.drop(columns=['expenditure']).columns, fill_value=0)

df = df.fillna(0)
df_case = df_case.fillna(0)

y = df['expenditure']
X = df.drop(columns=['expenditure'])

ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X, y)

r2 = r2_score(y, ridge.predict(X))

preds = ridge.predict(df_case)

result = pd.DataFrame({'client_name': df_case_id, 'pred_expenditure': preds.round(2)})
print("R² (Ridge):", round(r2, 3))
print(result)

R² (Ridge): 0.807
     client_name  pred_expenditure
0  Gary Williams            197.75
1    Jill Flores            -30.32
2   Johnny Jones            559.36


# Exercício 2. Classificação.

Implemente um modelo de `Regressão Logística` e um de `knn` para a classificação da classe `card` empregando os **4 atributos preditores com maior ganho de informação** (informação mútua, empregando `random_state=42` para calcular).

* **Valores ausentes**. Segue mesma lógica de substituição do exercício anterior, incluindo o `seed=42`.

* **Normalização**. Em ambos os casos normalize os dados com a normalização `Standard Scaler`.

* **Conjuntos de Treinamento e Teste**. Empregue 25% de dados de teste, mantendo nos conjuntos a mesma distribuição dos valores de `card` e `random_state=42`.

* **Outros parâmetros**. No **Knn**, empregue **6 vizinhos** mais próximos, e na **Regressão Logística**, `max_iter=1000`. Mantenha os demais parâmetros inalterados.





### 2.1. Quais os atributos com maior ganho de informação selecionados e quais são os seus valores?



In [25]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

df = pd.read_csv(path + 'credit_card_data.csv')

X = df.drop(columns=['card', 'client_name'])
y = df['card']

X = X.fillna(X.median(numeric_only=True))

X = pd.get_dummies(X, drop_first=True)

mi = mutual_info_classif(X, y, random_state=42)

mi_df = pd.DataFrame({'atributo': X.columns, 'mi': mi})
mi_df = mi_df.sort_values(by='mi', ascending=False).head(4)

print(mi_df)

      atributo        mi
4  expenditure  0.461230
3        share  0.395360
0      reports  0.099959
1          age  0.017681


### 2.2. Qual a acuracidade de cada um dos modelos obtidos e qual o **melhor modelo** (critério de acuracidade)?


In [28]:
df = pd.read_csv(path + 'credit_card_data.csv')

X = df.drop(columns=['card', 'client_name'])
y = df['card']

X = X.fillna(X.median(numeric_only=True))
X = pd.get_dummies(X, drop_first=True)

mi = mutual_info_classif(X, y, random_state=42)
mi_df = pd.DataFrame({'feat': X.columns, 'mi': mi}).sort_values('mi', ascending=False)
top4 = mi_df.head(4)['feat'].tolist()
X = X[top4]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)
sc = StandardScaler().fit(X_train)
Xs_train, Xs_test = sc.transform(X_train), sc.transform(X_test)

logit = LogisticRegression(max_iter=1000, random_state=42).fit(Xs_train, y_train)
knn   = KNeighborsClassifier(n_neighbors=6).fit(Xs_train, y_train)

acc_log = accuracy_score(y_test, logit.predict(Xs_test))
acc_knn = accuracy_score(y_test, knn.predict(Xs_test))

print(f"Acc Modelo 1 (LogisticRegression) = {acc_log:.3f}")
print(f"Acc Modelo 2 (KNN, k=6) = {acc_knn:.3f}")
print("Melhor modelo:", "LogisticRegression" if acc_log >= acc_knn else "KNN")

Acc Modelo 1 (LogisticRegression) = 0.927
Acc Modelo 2 (KNN, k=6) = 0.906
Melhor modelo: LogisticRegression



### 2.3. No **pior modelo** (critério de acuracidade), qual o valor de classe aprensenta mais Falso Positivos, e qual é essa quantidade?

In [32]:

y_pred_knn = knn.predict(X_test)

cm = confusion_matrix(y_test, y_pred_knn, labels=knn.classes_)

cm_df = pd.DataFrame(cm, index=knn.classes_, columns=knn.classes_)
print("Matriz de confusão (KNN):\n", cm_df, "\n")

fp = cm_df.sum(axis=0) - np.diag(cm_df)
print("Falsos positivos por classe:\n", fp, "\n")

worst_class = fp.idxmax()
worst_fp = int(fp.max())
print(f"Pior (mais FP): {worst_class}, FP = {worst_fp}")

Matriz de confusão (KNN):
         high  low  medium
high     251    0       0
low       74    0       0
medium     5    0       0 

Falsos positivos por classe:
 high      79
low        0
medium     0
dtype: int64 

Pior (mais FP): high, FP = 79


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


### 2.4. Qual a predição dos casos de `df_case_classification` empregando o melhor modelo? Qual a probalidade de cada valor de classe para a predição para `Eric Simon` e `Julian Choi`?



In [38]:
df = pd.read_csv(path + 'credit_card_data.csv')
df_case = pd.read_csv(path + 'credit_card_data_case_classification.csv')

case_names = df_case['client_name']

X = df.drop(columns=['card', 'client_name'])
y = df['card']

X = pd.get_dummies(X, drop_first=True)
X_case = pd.get_dummies(df_case.drop(columns=['client_name'], errors='ignore'), drop_first=True)

X_case = X_case.reindex(columns=X.columns, fill_value=0)

X = X.fillna(0)
X_case = X_case.fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_case_scaled = scaler.transform(X_case)

logreg = LogisticRegression(max_iter=1000, random_state=42)
logreg.fit(X_scaled, y)

preds = logreg.predict(X_case_scaled)
probs = logreg.predict_proba(X_case_scaled)

res = pd.DataFrame({'client_name': case_names, 'pred_class': preds})
probs_df = pd.DataFrame(probs, columns=logreg.classes_)
res = pd.concat([res, probs_df], axis=1)

out = res[res['client_name'].isin(['Eric Simon', 'Julian Choi'])]

for _, row in out.iterrows():
    print(f"{row['client_name']}, {row['pred_class']}")
    for c in logreg.classes_:
        print(f"{c} = {row[c]:.2f}", end=" ")
    print("\n")

Eric Simon, low
high = 0.37 low = 0.61 medium = 0.02 

Julian Choi, low
high = 0.08 low = 0.92 medium = 0.00 

